# Leverage Power of Machine Learning with ONNX

How to install .NET in Jupyter https://github.com/dotnet/interactive

## Instal Nuget Packages

In [ ]:
#r "nuget:Microsoft.ML,1.5.2"
#r "nuget:Microsoft.Data.Analysis,0.4.0"
#r "nuget: XPlot.Plotly, 4.0.3"
#r "nuget:Microsoft.ML.OnnxTransformer,1.5.2"
#r "nuget:Microsoft.ML.OnnxConverter,0.17.2"


Installed Packages fsharp.core, 4.7.2 XPlot.Plotly, 4.0.3

In [ ]:
using System.IO;
using Microsoft.Data.Analysis;
using XPlot.Plotly;

In [ ]:
using Microsoft.AspNetCore.Html;
using Microsoft.DotNet.Interactive.Formatting;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
Formatter.Register<DataFrame>((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }

    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));

    writer.Write(t);
}, "text/html");

## Load Data

In [ ]:
var data = DataFrame.LoadCsv("./SalaryData.csv");

In [ ]:
data

0,1.1,39343
1,1.3,46205
2,1.5,37731
3,2,43525
4,2.2,39891
5,2.9,56642
6,3,60150
7,3.2,54445
8,3.2,64445
9,3.7,57189
10,3.9,63218


In [ ]:
data.Description()

0,Length (excluding null values),30,30
1,Max,10.5,122391
2,Min,1.1,37731
3,Mean,5.3133335,76003


In [ ]:
static T[] Shuffle<T>(T[] array)
{
    Random rand = new Random();
    for (int i = 0; i < array.Length; i++)
    {
        int r = i + rand.Next(array.Length - i);
        T temp = array[r];
        array[r] = array[i];
        array[i] = temp;
    }
    return array;
}

int[] randomIndices = Shuffle(Enumerable.Range(0, (int)data.Rows.Count).ToArray());
int testSize = (int)(data.Rows.Count * .1);
int[] trainRows = randomIndices[testSize..];
int[] testRows = randomIndices[..testSize];

DataFrame data_train = data[trainRows];
DataFrame data_test = data[testRows];

display(data_train.Rows.Count);
display(data_test.Rows.Count);

27

3

## Use ML.NET
To use ML.NET, need to create instance of MLContext.

In [ ]:
using Microsoft.ML;

In [ ]:
var context = new MLContext();

## Create Pipeline

In [ ]:
#pragma warning disable CS1701
var pipeline = context.Transforms.Concatenate("Features", "yearsExperience")
                .Append(context.Regression.Trainers.Sdca(labelColumnName: "salary"));

## Training

In [ ]:
ITransformer model = pipeline.Fit(data_train);

## Evaluate

In [ ]:
var testResults = model.Transform(data_test);

var metrics = context.Regression.Evaluate(testResults, labelColumnName: "salary", scoreColumnName: "Score");

In [ ]:
Console.WriteLine($"*************************************************");
Console.WriteLine($"*       Metrics for regression model      ");
Console.WriteLine($"*------------------------------------------------");
Console.WriteLine($"*       LossFn:        {metrics.LossFunction:0.##}");
Console.WriteLine($"*       R2 Score:      {metrics.RSquared:0.##}");
Console.WriteLine($"*       Absolute loss: {metrics.MeanAbsoluteError:#.##}");
Console.WriteLine($"*       Squared loss:  {metrics.MeanSquaredError:#.##}");
Console.WriteLine($"*       RMS loss:      {metrics.RootMeanSquaredError:#.##}");
Console.WriteLine($"*************************************************");

*************************************************


*       Metrics for regression model      


*------------------------------------------------


*       LossFn:        56625073.33


*       R2 Score:      0.89


*       Absolute loss: 7453.15


*       Squared loss:  56625074


*       RMS loss:      7524.96


*************************************************


In [ ]:
private static readonly string MODEL_NAME = "model.onnx";

## Save ONNX Model

In [ ]:
using System.IO;

using (var stream = File.Create(MODEL_NAME))
{
    context.Model.ConvertToOnnx(model, data, stream);
}

## Fill in Input Data

![ONNX Netron Image](onnx-netron.PNG)

### That's all folks!